# Projeto

Uma grande empresa de tecnologia precisa de sua ajuda, eles foram hackeados! Felizmente, os engenheiros forenses coletaram dados valiosos sobre os hacks, incluindo informações como tempo da sessão, locais, velocidade de digitação do wpm etc. A engenheira forense relata a você o que ela foi capaz de descobrir até agora, ela conseguiu obter meta dados de cada sessão que os hackers usaram para se conectar aos seus servidores. Estas são as características dos dados:

* 'Session_Connection_Time': Quanto tempo a sessão durou em minutos
* 'Bytes Transferidos': Número de MB transferidos durante a sessão
* 'Kali_Trace_Used': indica se o hacker estava usando Kali Linux
* 'Servers_Corrupted': Número de servidores corrompidos durante o ataque
* 'Pages_Corrupted': Número de páginas acessadas ilegalmente
* 'Local': O ataque ao local veio (provavelmente inútil porque os hackers usaram VPNs)
* 'WPM_Typing_Speed': Sua velocidade de digitação estimada com base em logs de sessão.


A empresa de tecnologia tem 3 hackers em potencial que perpetraram o ataque. Eles têm certeza dos dois primeiros hackers, mas não têm muita certeza se o terceiro hacker estava envolvido ou não. Eles pediram sua ajuda! Você pode ajudar a descobrir se o terceiro suspeito teve ou não algo a ver com os ataques ou foram apenas dois hackers? Provavelmente não é possível saber com certeza, mas talvez o que você acabou de aprender sobre Clustering possa ajudar!

** Um último fato importante, o engenheiro forense sabe que os hackers compensam os ataques. O que significa que cada um deve ter aproximadamente a mesma quantidade de ataques. Por exemplo, se houvesse 100 ataques no total, em uma situação de 2 hackers, cada um deveria ter cerca de 50 hacks, em uma situação de três hackers cada um teria cerca de 33 hacks. O engenheiro acredita que esse é o elemento-chave para resolver isso, mas não sabe como distinguir esses dados não rotulados em grupos de hackers. **

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

# Carregando dados.
dataset = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [3]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [4]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [5]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [8]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [9]:
final_data = vec_assembler.transform(dataset)

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [12]:
# Calcule estatísticas resumidas ajustando o StandardScaler
scalerModel = scaler.fit(final_data)

In [13]:
# Normalize cada recurso para ter um desvio padrão da unidade.
cluster_final_data = scalerModel.transform(final_data)

** Hora de descobrir se são 2 ou 3! **

In [14]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [15]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [16]:
print("Com K=3")
print('--'*30)
print("Com K=2")


Com K=3
------------------------------------------------------------
Com K=2


Não há muito a ganhar com o WSSSE, afinal, esperaríamos que, conforme K aumenta, o WSSSE diminui. No entanto, poderíamos continuar a análise observando a queda de K = 3 para K = 4 para verificar se o agrupamento favorece números pares ou ímpares. Isso não será substancial, mas vale a pena dar uma olhada:

In [17]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
#    wssse = model.computeCost(cluster_final_data)
    print("With K={}".format(k))
#    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
------------------------------------------------------------
With K=3
------------------------------------------------------------
With K=4
------------------------------------------------------------
With K=5
------------------------------------------------------------
With K=6
------------------------------------------------------------
With K=7
------------------------------------------------------------
With K=8
------------------------------------------------------------


** Nada definitivo pode ser dito com o acima, mas espere! O último fato importante que o engenheiro mencionou foi que os ataques deveriam ser numerados igualmente entre os hackers! Vamos verificar as colunas de transformação e predição que resultam disso! Parabéns, se você fez essa conexão, foi bem complicado, considerando o que abordamos! **

In [18]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+



In [20]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



________

### Eram 2 hackers, na verdade, nosso algoritmo de clustering criou dois clusters do mesmo tamanho com K = 2, de jeito nenhum isso é uma coincidência!
